##  Optimze and a Dense Neural Network for gap filling and feature identification

** With a few tweaks to RepRunner, an LSTM can be run instead

In [18]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
# from functools import partial
from multiprocessing import Pool
from functools import partial
from sklearn import metrics
# from sklearn.model_selection import train_test_split

## Personal Modules
import ReadStandardTimeFill as RSTF
import importlib
import DenseNet as Dense
import MiscFuncs as MF
importlib.reload(Dense)
importlib.reload(RSTF)
importlib.reload(MF)

# %matplotlib inline

%matplotlib notebook
%config IPCompleter.greedy=True

from scipy.optimize import minimize, curve_fit
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.externals import joblib
from matplotlib import cm

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

from ipywidgets import FloatProgress, HTML
from IPython.display import display, clear_output
import os  
import shutil
from keras import backend as K
try:pool.close()
except:pass


def Test(params,X,y,YScaled,XScaled,pool):
    return(np.random.rand(params['K']))


def ModSelect(Scope,Site):
    if Site == 'Illisarvik':
        if Scope == 'Full':
            Model = ['wind_speed','air_pressure','PPFD_Avg','AirTC_Avg','VPD',
                    'Temp','VWC','Sedge','Shrub','Grass','Sparse','Out_of_Basin']
        if Scope == 'Test':
            Model = ['wind_speed','air_pressure','PPFD_Avg','AirTC_Avg','VPD',
                    'Temp','VWC','Sedge','Shrub','Grass','Sparse','Out_of_Basin']
#         if Scope == 'Test':
#             Model = ['PPFD_Avg','Sedge','Sparse']#,'VPD','wind_speed']
    if Site == 'FishIsland':
        BaseFactors = []
        if Scope == 'Full':
            Model = ['H','Wind Spd','air pressure','Ta','Rn','PPFD','Rain','Water Table',
            'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain','Wtr Tbl Trnd']
        if Scope == 'Test':
            Model = ['H','Water Table','Wind Spd','Active Layer']
    return(Model)

def Combos(Model,L,factor=None):
    Models=[]
    for c in combinations(Model,L):
        c = list(c)
        if factor is None:
            Models.append(c)
        else:
            for f in factor:
                f = f.split('+')
                if set(f).issubset(set(c)) and c not in Models:
                    Models.append(c)
                    
#     print('Models: ',Models)
    return(Models)

def Stats(mse,se,j,i,params):
    df = pd.DataFrame(index = [str(j)+'_'+str(i)],
                      data={'Model':[params['Model']],
                            'MSE':[mse],
                            'Size':j,
                            'Number':i,
                            'SE':[se],
                            'Performance':0})
    return(df)

def t(p,n):
    alpha = 1-p
    df = n-1
    return(stats.t.ppf(alpha,df))

FirstRun = True
FirstRun = False

MP=True
MP=False
processes=2
Scope = 'Full'
cwd = os.getcwd()
# for Site in ['Illisarvik','FishIsland']:
Site='Illisarvik'
target='fch4'
T= 1#t(0.05,params['K'])
print(cwd)


pd.set_option('max_colwidth',200)

C:\Users\wesle\NetworkAnalysis


# Model

In [19]:
def Display (tar,prog1,prog2,MdLs,MdL,kwt):
    clear_output()
    display(tar)
    display(prog1)
    display(prog2)
    display(MdLs)
    display(MdL)
    display(kwt)
    
tar = HTML(
            value=" ",
            placeholder='Target: ',
            description='Target: ',
        )
Time = time.time()

kwit = 0
if __name__ == '__main__':
    for target in ['fch4']:#['fco2','fch4']:
        if FirstRun == True:
            start=1
            stop = len(XVarriables)+1
            try:shutil.rmtree(cwd+'/'+Site+'/'+target+'/')
            except:pass
            os.mkdir(cwd+'/'+Site+'/'+target+'/')
        else:
            Records = pd.read_csv(cwd+'/'+Site+'/'+target+'_Runs.csv',delimiter = ',',header = 0,index_col=[0])
            start = 6#Records['Size'].max()+1
            Level = Records.loc[Records['Size']==start-1]
            Min = Level.loc[Level['MSE']==Level['MSE'].min()]
            Factors = Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Model'].values
            print(Level.sort_values('MSE'))
            Test = Min['Model'].values[0]
            stop = start + 1
            
        kwt = HTML(
                    value=str(0),
                    placeholder='Quit Score: ',
                    description='Quit Score: ',
                )
        tar.value=target
        XVarriables=ModSelect(Scope,Site)
        prog1 = FloatProgress(min=start, max=stop,description='Running:')
        prog2 = FloatProgress(min=0, max=100,description='Bootstrapping:')
        MdLs = HTML(
            value=" ",
            placeholder='Models: ',
            description='Models: ',
        )
        MdL = HTML(
            value=" ",
            placeholder='Testing: ',
            description='Testing: ',
        )
        Display (tar,prog1,prog2,MdLs,MdL,kwt)
        for j in range(start,stop):
            if j == 1: Inputs = (Combos(XVarriables,j))
            else: Inputs = (Combos(XVarriables,j,Factors))
            MdLs.value=str(len(Inputs))
            i = 0
            procede = True
            for Input in Inputs:
                if FirstRun == False:
                    print(Records.isin({'Size':[j]}).any().any())
                    if Records.loc[Records['Size']==j].isin({'Number':[i]}).any().any()==True:
                        procede = False
                    else:
                        procede = True
                if procede == True:
                    params = Dense.Params(Scope,target,MP)
                    params['Dpath'] = cwd+'/'+Site+'/'
                    params['Spath'] = params['Dpath']+'/'+target+'/'+str(j)+'_'+str(i)+'/'
                    try:os.mkdir(params['Spath'])
                    except:pass
                    params['Sname'] = 'Y_'
                    params['Inputs'] = Input
                    MdL.value='#'+str(i)+' '+str(params['Inputs'])
                    Display (tar,prog1,prog2,MdLs,MdL,kwt)
                    print(Level.sort_values('MSE'))
                    print('Runtime: ',time.time()-Time)
                    params['Model'] = '+'.join(params['Inputs'])
                    RST = RSTF.ReadStandardTimeFill(params,'ECData.csv')#,resample='2H')
                    RST.Scale(params['target'],params['Inputs']) 
                    y = RST.y*1.0
                    X = RST.X*1.0
                    params['N']=int(y.shape[0]/30)
                    params['Memory'] = (math.floor(100/params['proc'])- 5/params['proc']) * .01
                    Y_hat=[]
                    y_true=[]
                    X_true=[]
                    index=[]
                    ones=[]
                    prog2.value=0
                    if MP == False:
                        for k in range(params['K']):
                            Time2 = time.time()
                            results = Dense.Bootstrap(k,params,X,y)
                            Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                            y_true.append(RST.YScaled.inverse_transform(results[1]))
                            X_true.append(RST.XScaled.inverse_transform(results[2]))
                            ones.append(results[3])
                            prog2.value=(k+1)/params['K']*100
                            print(time.time()-Time2)
                    else:
                        pool = Pool(processes=processes,maxtasksperchild=75)
                        for k,results in enumerate(pool.imap(partial(Dense.Bootstrap,params=params,X=X,y=y),range(params['K']))):
                            Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                            y_true.append(RST.YScaled.inverse_transform(results[1]))
                            X_true.append(RST.XScaled.inverse_transform(results[2]))
                            ones.append(results[3])
                            prog2.value=(k+1)/params['K']*100
                        pool.close()
                    Y_hat = np.squeeze(np.asanyarray(Y_hat))
                    y_true = np.squeeze(np.asanyarray(y_true))
                    X_true = np.asanyarray(X_true)
                    ones = np.asanyarray(ones)
                    params['Memory'] = .95
                    if MP == False:
                        for k in range(1):
                             mse,se = Dense.Sort_outputs(k,params,Y_hat,y_true,X_true,ones)
                    else:
                        pool = Pool(processes=1,maxtasksperchild=75)
                        for k,results in enumerate(pool.imap(partial(Dense.Sort_outputs,params=params,
                         Y_hat=Y_hat,y_true=y_true,X_true=X_true,ones=ones),range(1))):
                             mse,se = results
                        pool.close()
                    if i == 0 and FirstRun == True:Level = Stats(mse,se,j,i,params)
                    else:
                        Level = Level.loc[Level['Size']==j]
                        Level = Level.append(Stats(mse,se,j,i,params))
                i += 1
                prog1.value=j+i/len(Inputs)
            Min = Level.loc[Level['MSE']==Level['MSE'].min()]
            Factors = Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Model'].values
            Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Performance']=1
            if j == 1:
                Records = Level
            else:
                if Min['MSE'].values[0]+Min['SE'].values[0]*T > Records['MSE'].min():
                    kwit += 1
                    print('Stop Score: ',kwit)
                elif kwit > 0:
                    kwit -= 1
                    print('Stop Score: ',kwit)
                Records = Records.append(Level)
            kwt.value = str(kwit)
            if Scope == 'Full':
                Records = Records.reset_index(drop=True)
                Records.drop(Records.columns[Records.columns.str.contains('Unnamed',case = False)],axis = 1)
                Records.to_csv(params['Dpath']+'/'+target+'_Runs.csv')
            else:
                Records = Records.reset_index(drop=True)
                Records.drop(Records.columns[Records.columns.str.contains('Unnamed',case = False)],axis = 1)
                Records.to_csv(params['Dpath']+'/'+target+'_Runs2.csv')
                print(Level)
            if kwit >= 2:
                break

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

           MSE                                                     Model  \
6_19  0.111737              wind_speed+air_pressure+Temp+VWC+Sedge+Shrub   
6_17  0.112706                wind_speed+air_pressure+VPD+Temp+VWC+Grass   
6_12  0.113621  wind_speed+air_pressure+AirTC_Avg+VWC+Sedge+Out_of_Basin   
6_16  0.113713                wind_speed+air_pressure+VPD+Temp+VWC+Sedge   
6_14  0.113947        wind_speed+air_pressure+AirTC_Avg+VWC+Grass+Sparse   
6_23  0.114479              wind_speed+air_pressure+Temp+VWC+Shrub+Grass   
6_13  0.115575         wind_speed+air_pressure+AirTC_Avg+VWC+Shrub+Grass   
6_20  0.115906              wind_speed+air_pressure+Temp+VWC+Sedge+Grass   
6_15  0.116322  wind_speed+air_pressure+AirTC_Avg+VWC+Grass+Out_of_Basin   
6_18  0.116465        wind_speed+air_pressure+VPD+VWC+Sedge+Out_of_Basin   
6_25  0.116498       wind_speed+air_pressure+Temp+VWC+Grass+Out_of_Basin   
6_22  0.117402       wind_speed+air_pressure+Temp+VWC+Sedge+Out_of_Basin   
6_24  0.1222

KeyboardInterrupt: 

# Map Results

In [20]:
print(Level)

Records = Records.append(Level)
kwt.value = str(kwit)
if Scope == 'Full':
    Records = Records.reset_index(drop=True)
    Records.drop(Records.columns[Records.columns.str.contains('Unnamed',case = False)],axis = 1)
    Records.to_csv(params['Dpath']+'/'+target+'_Runs.csv')
else:
    Records = Records.reset_index(drop=True)
    Records.drop(Records.columns[Records.columns.str.contains('Unnamed',case = False)],axis = 1)
    Records.to_csv(params['Dpath']+'/'+target+'_Runs2.csv')
print(Records)
if kwit >= 2:
    break

           MSE                                                     Model  \
6_12  0.113621  wind_speed+air_pressure+AirTC_Avg+VWC+Sedge+Out_of_Basin   
6_13  0.115575         wind_speed+air_pressure+AirTC_Avg+VWC+Shrub+Grass   
6_14  0.113947        wind_speed+air_pressure+AirTC_Avg+VWC+Grass+Sparse   
6_15  0.116322  wind_speed+air_pressure+AirTC_Avg+VWC+Grass+Out_of_Basin   
6_16  0.113713                wind_speed+air_pressure+VPD+Temp+VWC+Sedge   
6_17  0.112706                wind_speed+air_pressure+VPD+Temp+VWC+Grass   
6_18  0.116465        wind_speed+air_pressure+VPD+VWC+Sedge+Out_of_Basin   
6_19  0.111737              wind_speed+air_pressure+Temp+VWC+Sedge+Shrub   
6_20  0.115906              wind_speed+air_pressure+Temp+VWC+Sedge+Grass   
6_21  0.124296             wind_speed+air_pressure+Temp+VWC+Sedge+Sparse   
6_22  0.117402       wind_speed+air_pressure+Temp+VWC+Sedge+Out_of_Basin   
6_23  0.114479              wind_speed+air_pressure+Temp+VWC+Shrub+Grass   
6_24  0.1222

SyntaxError: 'break' outside loop (<ipython-input-20-0bddfddcc274>, line 15)

# Predict

In [ ]:

plt.figure(figsize=(8,7))
Data = Data.sort_values(by=Key)
# Data.index = Data[Key]

Data = Data[np.isfinite(Data['True'])]

plt.scatter(RST.Master[Key],RST.Master[target],edgecolor='black',facecolor='white')
plt.plot(Data[Key],Data[target],
         label= params['target']+' Model\nr^2: '+str(np.round(metrics.r2_score(Data['True'],
                                                                   Data[params['target']])**2,3)))
# plt.plot(Data.index,Data['Var'],label= params['target']+
# ' Model\nRMSE: '+str(np.round(metrics.mean_squared_error(Data['True'],
#                                                                    Data[params['target']])**2,3)))


plt.fill_between(Data[Key], Data[target]-Data['PI'], 
                 Data[target]+Data['PI'],  color = 'green', alpha = 0.4, 
                 label = '95% PI')
plt.fill_between(Data[Key], Data[target]-Data['CI'], 
                 Data[target]+Data['CI'],  color = 'red', alpha = 0.4, 
                 label = '95% CI')
plt.legend()
# print(RST.Master)

plt.figure()

# plt.plot(Data[Key], Data['Var'])

plt.scatter(Data[target],Data['True'])

# print(Data['SE'])